In [0]:
from pyspark.sql import SparkSession

In [0]:
spark= SparkSession.builder.appName('build').getOrCreate()

In [0]:
df=spark.read.csv('/FileStore/tables/hack_data.csv',inferSchema=True,header=True)

In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StringIndexer, StandardScaler

df.columns

Out[36]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],outputCol='features')

In [0]:
output = assembler.transform(df)

In [0]:
scaler = StandardScaler(inputCol='features',outputCol='scaled_feat')
scaler_mod=scaler.fit(output)
fin_df=scaler_mod.transform(output)

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

k_clust2=KMeans(featuresCol='scaled_feat',k=2)
k_clust3=KMeans(featuresCol='scaled_feat',k=3)
k_mod_2=k_clust2.fit(fin_df)
k_mod_3=k_clust3.fit(fin_df)

In [0]:
k_mod_2.transform(fin_df).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [0]:
k_mod_3.transform(fin_df).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+